In [122]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [101]:
!pip install pm4py

In [0]:
!ls '/content/drive/My Drive/Eventlog/Benchmark/seperate file/'


 BPIC2018	   Parbablitiy_Based		     TestFile
'concatenation '   Probability_ab_50		     TestFile.xes
 final_XES	   Prob_SEPSIS_Second_Method	    'with removing infrequent'
 Freq_changes	   Prob_SIPSIS_Second_Selfloop_end   WRI
 Frequency_Based   Prob_TwoThreshold_SameThreshold   XES_All


In [0]:
#Importing library to read log file
from pm4py.objects.log.importer.xes import factory as xes_import_factory
from pm4py.objects.log.exporter.csv import factory as csv_exporter
from pm4py.objects.conversion.log import factory as conversion_factory
from pm4py.objects.log.exporter.xes import factory as xes_exporter
from pm4py.algo.filtering.pandas.variants import variants_filter

from datetime import datetime
import pandas as pd
import operator
import re

In [0]:
#Reading XES fileafter 

filename = '/content/drive/My Drive/Eventlog/Benchmark/seperate file/TestFile.xes'
log = xes_import_factory.apply(filename)

#Converting XES to CSV file
fileName = "/content/drive/My Drive/Eventlog/Benchmark/seperate file/TestFile.csv"

csv_exporter.export(log, fileName)

# Reading the Converted CSV file
df_sepsis = pd.read_csv(fileName)


### Reading dataset 

In [104]:
df_sepsis.head()

case:concept:name concept:name             time:timestamp
0                 A          A01  2014-10-22 11:15:41+02:00
1                 A          A02  2014-10-22 11:27:00+02:00
2                 A          A03  2014-10-22 11:27:00+02:00
3                 A          A04  2014-10-22 11:27:00+02:00
4                 A          A05  2014-10-22 11:33:37+02:00

### Length of dataset

In [105]:
len(df_sepsis)

254

### List of Unique activity

In [106]:
unique_activity = df_sepsis["concept:name"].unique()
unique_activity   

array(['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09',
       'A10', 'A11', 'A12', 'A13'], dtype=object)

### Number of distinct trace

In [0]:
# Save all trace id and activity belongs to a trace into dictionary
def activity(df_eve,unique_id):
  traces = {}
  for i in unique_id:
    list_a = []
    row_filter = df_eve.loc[df_eve['case:concept:name']==i,:]
    row_filter  = row_filter.reset_index(drop=True)
    for n in range(len(row_filter)):
      list_a.append(row_filter.loc[row_filter['case:concept:name']==i,'concept:name'][n])
    traces[i] = list_a
  return traces


# Filtering Distinct trace only
def dist_trace(df_eve,unique_id):
  
  Distinct_trace = []
  Distinct_trace_id = []
  # Calling activity method
  traces_list = activity(df_eve,unique_id)
  
  for k, v in traces_list.items():
      if v not in Distinct_trace:
        Distinct_trace.append(v)
        Distinct_trace_id.append(k)
        
  return Distinct_trace_id,Distinct_trace

unique_id = df_sepsis["case:concept:name"].unique()
dist_trace_id, dist_traces = dist_trace(df_sepsis,unique_id)

In [9]:
len(dist_traces)

20

### Calculating frequency of each activity

In [0]:
# Frequency of single activity
def activity_count(df_eve,unique_id,fre_act):
  
  Freq_act = 0
  for i in unique_id:
    row_filter = df_eve.loc[df_eve['case:concept:name']==i,:]
    row_filter  = row_filter.reset_index(drop=True)
    
    if fre_act:
      len_act = len(row_filter.loc[row_filter["concept:name"]==fre_act,"concept:name"])
      Freq_act +=len_act
    
  return Freq_act

act_rename = unique_activity
freq_activity = {}
for i in act_rename:
  frequency_act = activity_count(df_sepsis,unique_id,i)
  freq_activity[i] = frequency_act


In [0]:
freq_activity

### All possible activity combinations  

In [0]:
# The following functions gets combination of events
def combination_of_activity(activity_name):
  
  #activity_name = act_rename
  
  # Combine two two activity and save into activity name
  Activity_Name = []
  for i in range(len(activity_name)):
    for c in range(len(activity_name)):
      Activity_Name.append([activity_name[i],activity_name[c]])
      #Activity_Name.append([activity_name[c],activity_name[i]])
        
  # Save unique combination only
  Activity_combination = []
  for combination in Activity_Name:
    if combination not in Activity_combination:
      Activity_combination.append(combination)
      
  return Activity_combination

  
Activity_combi = combination_of_activity(act_rename)
Activity_combi

In [13]:
len(Activity_combi)

169

In [0]:
# The following functions gets combination of events
def combination_of_activity(activity_name):
  
  #activity_name = act_rename
  
  # Combine two two activity and save into activity name
  Activity_Name = []
  for i in range(len(activity_name)):
    for c in range(len(activity_name)):
      if [activity_name[i],activity_name[c]] not in Activity_Name:
        Activity_Name.append([activity_name[i],activity_name[c]])
      
      
  return Activity_Name

  
Activity_combi = combination_of_activity(act_rename)
Activity_combi

In [0]:
len(Activity_combi)

169

### Calculating frequency of each combination of activity

In [0]:
# Count the combination
# The function gets trace list and activity combination as input
def combination_counter(trace_list,activity_combination):
  # The combination of activity is key and value is number of times the combination comimg together
  counts_of_combination = {}
  
  # Each activity goes into the loop one by one 
  for trace_activity in trace_list:
  
    combination_of_observation = []
    # Creating the combination and save it into Activity name 
    for i in range(len(trace_activity)-1):
        combination_of_observation.append([trace_activity[i],trace_activity[i+1]])
        
    # Checking whether the resulted combination is presented or not in the activity combination list
    # When a resulted combination is presented in activity combination, the resulted combination is saved into 
    # counts of combination as key and number of times it is presented into the unique trace saved into 
    # counts of combination as value of it
    for c in combination_of_observation:
      if c in activity_combination:
        k =  repr(c)
        if k not in counts_of_combination.keys():
          counts_of_combination[k] = 1
        else:
          counts_of_combination[k] += 1
    
  return counts_of_combination
CountsofComb = combination_counter(dist_traces,Activity_combi)
CountsofComb

### Calculating probability of combination for (ab|a) and (ab|b)

In [0]:
# The following functions finds probability of each combination of activity given activity
def prob_of_activity(CountsofComb,freq_activity,threshold_value):
  # Dictionary "ab_a" to save the probability of each combination of activity given first activity
  # Dictionary "ab_b" to save the probability of each combination of activity given second activity
  # Dictionary "similar" to save the probability of "ab_a" and "ab_b" interaction
  ab_a = {}
  ab_b = {}
  similar = {}

  #sorted_d = dict( sorted(CountsofComb.items(), key=operator.itemgetter(1),reverse=True))

  # Finding the probability of each combination of activity given first activity
  for k , v in CountsofComb.items():
    if k[2:5] in freq_activity:
      prob = round((v / freq_activity[k[2:5]]),3)  
      if prob >= threshold_value:
        a = repr([k[2:5],k[9:12]])
        ab_a[a] = prob


  # Finding the probability of each combination of acitivity given second activity
  for k , v in CountsofComb.items():
    if k[2:5] in freq_activity:
      prob = round((v / freq_activity[k[9:12]]),3)
      if prob >= threshold_value:
        b = repr([k[2:5],k[9:12]])
        ab_b[b] = prob
     

  for i in ab_a.keys():
    if i in ab_b.keys():
      c = repr([i[2:5],i[9:12]])
      prob_add = ab_a[i] + ab_b[i]
      similar[c] = round(prob_add,2)

  similar = dict( sorted(similar.items(), key=operator.itemgetter(1),reverse=True))
  similar_keys = list(similar.keys())
  similar_values = similar.values()
  similar = reorder_activitylist(similar)
  
  return similar

#### Probability cut off value is 0.5

In [0]:
#ab_a, ab_b, similar_activity,similar_prob = prob_of_activity(CountsofComb,freq_activity,0.1)
similar_activity = prob_of_activity(CountsofComb,freq_activity,0.1)

### Probability of (ab | a)

In [0]:
ab_a

### Probability of (ab | b)

In [0]:
ab_b 

### Addition of Union of Probability of combination 

In [0]:
def reorder_activitylist(similar):
  prob_p = []
  prob_a = []
  for a, p in similar.items():
    prob_a.append(a)
    prob_p.append(p)


  counter=collections.Counter(prob_p)
  prob_morethanone = []
  for prob, n in counter.items():
    if n > 1:
      prob_morethanone.append(prob)

  for p in prob_morethanone:
    n = 0
    while counter[p] != n:

      prob_activity = []
      prob_activity_key = []
      for k,v in enumerate(prob_p):
        if v == p:
          prob_activity.append(prob_a[k])
          prob_activity_key.append(k)

      max_value,index_ = frequncyOfposition(prob_activity)
      final_order = reorder_activity(prob_activity,max_value,index_)

      new_key = 0
      for k in prob_activity_key:
        prob_a[k] = final_order[new_key]
        new_key +=1
      break
  return prob_a

In [0]:
def indexCounterOfactivity(df_eve, activity_name):
  indexOf_combination_activity = []
  # Taking unique number of trace  
  unique_id = df_eve["case:concept:name"].unique()
  # Filter all unique trace one by one
  for i in unique_id:
    # Save the index of matched activity
    event_index = []
    # Filter one trace at a time
    row_filter = df_eve.loc[df_eve['case:concept:name']==i,:]
    # Reset the index of the filtered trace
    row_filter  = row_filter.reset_index(drop=True)
    
    # The activity of the trace will be combared with desired combination activity
    # if it matches occured, the index of activity is saved into event index
    for event in range(len(row_filter.loc[:,'concept:name'])-1):
      # if The two activity is identical in the combination activity, the following condition will be excuted  
      if activity_name[2:5] == activity_name[9:12]:
        if (activity_name[2:5] == row_filter.loc[event,'concept:name']) & (activity_name[9:12] == row_filter.loc[event + 1,'concept:name']):
          # The index of the activity will be appended into event index
          event_index.append(event)
          event_index.append(event + 1)
          
      # If the two activity is not identical in the combination activity, the following will be excuted
      elif (activity_name[2:5] == row_filter.loc[event,'concept:name']) & (activity_name[9:12] == row_filter.loc[event + 1,'concept:name']):
        # The index of the activity will be appended into index
        event_index.append(event)
        event_index.append(event + 1)

    if len(event_index) !=0:
      indexOf_combination_activity.append(event_index)
    
  return indexOf_combination_activity

In [0]:
def frequncyOfposition(prob_activity):

  max_value = []
  index_ = []
  for l in prob_activity:
    max_value_i = []
    index_i = []
    df = indexCounterOfactivity (df_sepsis, l)

    activity_counts = {}
    for a in df:

      if len(a) >2:
        a_ = []
        for a_i in range(0,len(a)-1):
          a_.append([a[a_i],a[a_i+1]])

        for i in a_:
          i = repr(i)
          if i not in activity_counts.keys():
            activity_counts[i] = 1
          else:
            activity_counts[i] += 1 

      else:
        a = repr(a)
        if a not in activity_counts.keys():
          activity_counts[a] = 1
        else:
          activity_counts[a] += 1 

    for k, v in activity_counts.items(): 
      if v == max(activity_counts.values()):
        max_value_i.append(v)
        index_i.append(k)
        
        
    if len(max_value_i)>1:
      final_order = reorder_activity1(max_value_i,index_i)  
      max_value.append(max_value_i[final_order[0]])
      index_.append(index_i[final_order[0]])
    else:
      max_value.append(max_value_i[0])
      index_.append(index_i[0])

  return max_value, index_

In [0]:
def reorder_activity1(max_value,index_):

  index_max_value = sorted(max_value,reverse=True)
  # convert string data types to int
  index_new = []
  for i in index_:
    num = re.findall(r'\d+', i) 
    result = list(map(int, num))
    index_new.append(result)


  # Finding the equal frequency of index and save into empty set
  # Finding index of all value
  max_value_new = set()
  final_index = []
  for i in range(len(index_max_value)-1):
    if index_max_value[i] == index_max_value[i+1]:
      max_value_new.add(i)
      final_index.append(i)
      max_value_new.add(i+1)
      final_index.append(i+1)
    else:
      final_index.append(i)
  
  if len(max_value_new)>0:
    # Compare position of equal frequency by finding minimum value of in the position 
    new_value = {}
    for i in range(list(max_value_new)[0],len(max_value_new)):
      new_value[i] = min(index_new[i])

    # the final_index will be updated
    index_max_value1 = sorted(list(new_value.values()))
    fi = 0
    for i in index_max_value1:
      for k,v in new_value.items():
         if i == v:
            final_index[list(max_value_new)[fi]] = k
            fi +=1

    return final_index


In [0]:
def reorder_activity(prob_activity,max_value,index_):

  index_max_value = sorted(max_value,reverse=True)
  # convert string data types to int
  index_new = []
  for i in index_:
    num = re.findall(r'\d+', i) 
    result = list(map(int, num))
    index_new.append(result)


  # Finding the equal frequency of index and save into empty set
  # Finding index of all value
  max_value_new = set()
  final_index = []
  for i in range(len(index_max_value)-1):
    if index_max_value[i] == index_max_value[i+1]:
      max_value_new.add(i)
      final_index.append(i)
      max_value_new.add(i+1)
      final_index.append(i+1)
    else:
      final_index.append(i)
  
  if len(max_value_new) >0:
    # Compare position of equal frequency by finding minimum value of in the position 
    new_value = {}
    for i in range(list(max_value_new)[0],len(max_value_new)):
      new_value[i] = min(index_new[i])

    # the final_index will be updated
    index_max_value1 = sorted(list(new_value.values()))
    fi = 0
    for i in index_max_value1:
      for k,v in new_value.items():
         if i == v:
            final_index[list(max_value_new)[fi]] = k
            fi +=1

    # The order of filtered activity will be changed
    final_order = []
    for k in final_index:
      final_order.append(prob_activity[k])

    return final_order 
  else:
    
    final_order = []
    for k in index_max_value:
      k = index_max_value.index(k)
      fk = max_value.index(index_max_value[k])
      final_order.append(prob_activity[fk]) 

    return final_order

### Concatenate the event based on highest probability of combination

In [0]:
def event_dropper(df_eve, activity_name, new_activity_name):
  # Taking unique number of trace  
  unique_id = df_eve["case:concept:name"].unique()
  # Creating empty dataframe
  df_final = pd.DataFrame(columns=df_eve.columns)
  
  # Filter all unique trace one by one
  for i in unique_id:
    # Save the index of matched activity
    event_index = []
    # Filter one trace at a time
    row_filter = df_eve.loc[df_eve['case:concept:name']==i,:]
    # Reset the index of the filtered trace
    row_filter  = row_filter.reset_index(drop=True)
    
    # The activity of the trace will be combared with desired combination activity
    # if it matches occured, the index of activity is saved into event index
    for event in range(len(row_filter.loc[:,'concept:name'])-1):
      # if The two activity is identical in the combination activity, the following condition will be excuted  
      if activity_name[2:5] == activity_name[9:12]:
        if (activity_name[2:5] == row_filter.loc[event,'concept:name']) & (activity_name[9:12] == row_filter.loc[event + 1,'concept:name']):
          # The index of the activity will be appended into event index
          event_index.append(event)
          event_index.append(event + 1)
          
      # If the two activity is not identical in the combination activity, the following will be excuted
      elif (activity_name[2:5] == row_filter.loc[event,'concept:name']) & (activity_name[9:12] == row_filter.loc[event + 1,'concept:name']):
        # The index of the activity will be appended into index
        event_index.append(event)
        row_filter.loc[event + 1,'concept:name']=row_filter.loc[event + 1,'concept:name'].replace(activity_name[9:12],new_activity_name)
       
    # The list of the saved index of activity will be dropped from the trace
    row_filter.drop(row_filter.index[event_index],inplace=True)
    # The resulting trace will be merged with new dataframe 
    df_final = pd.concat([df_final, row_filter],ignore_index=True)

  return df_final

### Removing self loop

In [0]:
def remove_self_loop(df,file_Num,replace_activity):
     
  same_act = []
  act_rename = df["concept:name"].unique()
  unique_id = df["case:concept:name"].unique()
  dist_trace_id, dist_traces = dist_trace(df,unique_id)
  Activity_combi = combination_of_activity(act_rename)
  CountsofComb1 = combination_counter(dist_traces,Activity_combi)

  sorted_df = dict( sorted(CountsofComb1.items(), key=operator.itemgetter(1),reverse=True))
  for k , v in sorted_df.items():
    #print(k, ":", v)
    if k[2:5]==k[9:12]:
      same_act.append(k)
      #print(k, ":", v)
  
  new_a = same_act[0]
  df1 = event_dropper(df, new_a, None)
  for a in range(1,len(same_act)):
    new_ac = same_act[a]
    df1 = event_dropper(df1,new_ac,None)
  
  
  csv_file = '/content/drive/My Drive/Eventlog/Benchmark/seperate file/Test/TestFile_After_removingSelfLoop.csv'
  df1.to_csv(csv_file,index=False)
  # Converting csv file to log file
  log = conversion_factory.apply(df1)
  # Saving log into xes file
  xes_file = '/content/drive/My Drive/Eventlog/Benchmark/seperate file/Test/TestFile_{}.xes'.format(file_Num)
  xes_exporter.export_log(log, xes_file)


### Feed different cut off probability to get number of XES files

In [0]:
import numpy as np
threshold_values = np.arange(0.1, 1, 0.1)
n = 1
replace_activity = {}

for t_v in threshold_values:
  similar_activity = prob_of_activity(CountsofComb,freq_activity,t_v)


  if len(similar_activity)>=1:
    
    # first combination
    if similar_activity[0][2:5] == similar_activity[0][9:12]:      
      df = event_dropper(df_sepsis, similar_activity[0],None)
    else:
      df = event_dropper(df_sepsis, similar_activity[0],"BBB")
      for ak in [similar_activity[0][2:5], similar_activity[0][9:12]]: 
        if ak not in replace_activity.keys():
          replace_activity[ak] = "BBB"
    
    # Remaining combination
    for i in range(1,len(similar_activity)):
      if similar_activity[i][2:5] == similar_activity[i][9:12]:
        df = event_dropper(df, similar_activity[i],None)
      else:
        df = event_dropper(df, similar_activity[i],"BB{}".format(i))
        for ak in [similar_activity[i][2:5], similar_activity[i][9:12]]:
          if ak not in replace_activity.keys():
            replace_activity[ak] = "BB{}".format(i)
            
  for rk in replace_activity.keys():
    df["concept:name"] = df["concept:name"].apply(lambda x: re.sub( rk, replace_activity[rk] , x))
  
  remove_self_loop(df,n,replace_activity)
  n +=1
  

In [0]:
csv_file = '/content/drive/My Drive/Eventlog/Benchmark/seperate file/TestFile_After_concat.csv'
df.to_csv(csv_file,index=False)

In [0]:
# Converting csv file to log file
log = conversion_factory.apply(df)
# Saving log into xes file
xes_file = '/content/drive/My Drive/Eventlog/Benchmark/seperate file/TestFile_After_concat.xes'
xes_exporter.export_log(log, xes_file)

### After concatenation and Removing self loop, result of the dataset

In [0]:
#Reading XES fileafter 

filename = '/content/drive/My Drive/Eventlog/Benchmark/seperate file/TestFile_1.xes'
log = xes_import_factory.apply(filename)

#Converting XES to CSV file
fileName = "/content/drive/My Drive/Eventlog/Benchmark/seperate file/no_need8.csv"

csv_exporter.export(log, fileName)

# Reading the Converted CSV file
df_sepsis_5 = pd.read_csv(fileName)

In [0]:
df_sepsis_5.head()

case:concept:name concept:name             time:timestamp
0                 A          BBB  2014-10-22 11:15:41+02:00
1                 A          A02  2014-10-22 11:27:00+02:00
2                 A          A03  2014-10-22 11:27:00+02:00
3                 A          A04  2014-10-22 11:27:00+02:00
4                 A          BB1  2014-10-22 11:34:00+02:00

### Length of the dataset

In [0]:
len(df_sepsis_5)

197

### Unique activity

In [0]:
unique_activity = df_sepsis_5["concept:name"].unique()
unique_activity 

array(['BBB', 'A02', 'A03', 'A04', 'BB1', 'BB2', 'A09', 'A10', 'A11',
       'A12', 'A13'], dtype=object)

In [0]:
len(unique_activity)

11

### Number of distinct trace (0.5)

In [0]:
unique_id = df_sepsis_5["case:concept:name"].unique()
dist_trace_id, dist_traces = dist_trace(df_sepsis_5,unique_id)

In [0]:
len(dist_traces)

20